In [1]:
import pandas as pd
import glob
from datetime import datetime
import pandas as pd
import requests #Pushshift accesses Reddit via an url so this is needed
import json #JSON manipulation
import csv #To Convert final table into a csv file to save to your machine
import time
import datetime as dt
import numpy as np


df_mean=pd.read_csv("/Users/manbubble/Downloads/RedditAnalize/BitcoinGet/t3t6_mean.csv",encoding="ISO-8859-1")
df_abs=pd.read_csv("/Users/manbubble/Downloads/RedditAnalize/BitcoinGet/t3t6_abs_mean.csv",encoding="ISO-8859-1")
df_var=pd.read_csv("/Users/manbubble/Downloads/RedditAnalize/BitcoinGet/t3t6_var.csv",encoding="ISO-8859-1")
df_Bitcoin=pd.read_csv("/Users/manbubble/Downloads/RedditAnalize/BitcoinGet/timesortBitcoin.csv",encoding="ISO-8859-1")
# 対数差をビットコンでとって、pdにする。差の基準のtimeスタンプ持っておいて、同じタイムスタンプをもつt3とt6に連結
#df_Bitcoin.diff()で差を取る　そしてlog np.log(df)でとれる
#df_Bitcoinはopenだけ取り出したほうがいい、最初の１行目のNAの行は削除する
# そして、timestampは2021年11月1日00:00の行を除いて連結

print(df_mean.dropna(subset=['6_hours_ago']))
print(df_mean.dropna(subset=['3_hours_ago']))
# これでsubsetに指定した列がNanである列を取り除く


                      time  6_hours_ago  3_hours_ago
10     2018-04-01 10:00:00     0.000000     0.000000
11     2018-04-01 11:00:00     0.000000     0.000000
12     2018-04-01 12:00:00     0.000000     0.000000
13     2018-04-01 13:00:00     0.200000     0.125000
14     2018-04-01 14:00:00     0.166667     0.100000
...                    ...          ...          ...
31436  2021-10-31 20:00:00    -0.250000    -0.250000
31437  2021-10-31 21:00:00     0.000000    -0.200000
31438  2021-10-31 22:00:00     0.000000    -0.200000
31439  2021-10-31 23:00:00    -0.200000    -0.222222
31440  2021-11-01 00:00:00    -0.200000    -0.111111

[22124 rows x 3 columns]
                      time  6_hours_ago  3_hours_ago
10     2018-04-01 10:00:00     0.000000     0.000000
11     2018-04-01 11:00:00     0.000000     0.000000
12     2018-04-01 12:00:00     0.000000     0.000000
13     2018-04-01 13:00:00     0.200000     0.125000
14     2018-04-01 14:00:00     0.166667     0.100000
...                 

In [20]:
# len(df_Bitcoin)
# df_Bitcoin.dropna(subset=['conversionType'])
# df_Bitcoin[df_Bitcoin.duplicated(subset='time')]
df_Bitcoin=df_Bitcoin.drop_duplicates(subset='time')
# timeの重複を取り除く

# df_Bitcoin['open'].diff()

In [22]:
# いらない列を削除する


df_Bitcoin=df_Bitcoin.drop(['close', 'conversionType','conversionSymbol'], axis=1)

In [32]:
import numpy as np

Bitcoin_log=np.log(df_Bitcoin.loc[:,['high','low','open','volumefrom']])
Bitcoin_log_diff=Bitcoin_log.loc[:,['high','low','open','volumefrom']].diff()

In [44]:
df2=Bitcoin_log_diff[1:]
# 最初の行を削除 対数さをとったやつの最初はNanだから
df2=df2.reset_index(drop=True)


In [45]:
df1=df_Bitcoin['time'][:-1]
df1=df1.reset_index(drop=True)
# インデックスを振り直す
# これで最後の行を削除

In [48]:

df_h = pd.concat([df1, df2], axis=1)
df_h
# timeと対数差を整えて連結

,time,high,low,open,volumefrom
0,2018-04-01 00:00:00,0.013297,0.004177,-0.002860,0.683997
1,2018-04-01 01:00:00,0.000096,0.001754,0.013493,-0.037853
2,2018-04-01 02:00:00,-0.004781,-0.004786,-0.004023,-0.237691
3,2018-04-01 03:00:00,-0.009421,-0.005332,-0.011961,-0.132356
4,2018-04-01 04:00:00,0.000061,-0.014133,0.003478,0.751382
...,...,...,...,...,...
31435,2021-10-31 19:00:00,-0.002938,-0.001075,0.001923,-0.209830
31436,2021-10-31 20:00:00,-0.001648,-0.002445,-0.006050,0.114681
31437,2021-10-31 21:00:00,-0.002891,-0.002894,-0.002556,0.167903
31438,2021-10-31 22:00:00,0.001945,0.003239,0.001513,0.017864


In [58]:
df3=df_mean[:-1].loc[:,['6_hours_ago','3_hours_ago']]

df_Bitcoin_sentiment=pd.concat([df_h,df3],axis=1)
df_Bitcoin_sentiment.to_csv("complete_mean_exist_Nan.csv",index=False)

In [59]:
df_mean_ommit_na = df_Bitcoin_sentiment.dropna(how='any')
# nanがあればすべて取り除く

In [60]:
df_mean_ommit_na

,time,high,low,open,volumefrom,6_hours_ago,3_hours_ago
10,2018-04-01 10:00:00,-0.003835,-0.011489,0.002870,0.267168,0.000000,0.000000
11,2018-04-01 11:00:00,-0.007720,0.000186,-0.010919,-0.159632,0.000000,0.000000
12,2018-04-01 12:00:00,0.001321,0.003349,-0.000236,-0.295588,0.000000,0.000000
13,2018-04-01 13:00:00,-0.000447,-0.001616,0.002223,0.180988,0.200000,0.125000
14,2018-04-01 14:00:00,-0.005130,-0.012716,-0.001908,0.954167,0.166667,0.100000
...,...,...,...,...,...,...,...
31435,2021-10-31 19:00:00,-0.002938,-0.001075,0.001923,-0.209830,-0.333333,-0.166667
31436,2021-10-31 20:00:00,-0.001648,-0.002445,-0.006050,0.114681,-0.250000,-0.250000
31437,2021-10-31 21:00:00,-0.002891,-0.002894,-0.002556,0.167903,0.000000,-0.200000
31438,2021-10-31 22:00:00,0.001945,0.003239,0.001513,0.017864,0.000000,-0.200000


In [61]:
df_mean_ommit_na.to_csv("omitNan_mean.csv",index=False)
# 解析ではこれを使う

In [64]:
# ここから下は分散
df4=df_var[:-1].loc[:,['6_hours_ago','3_hours_ago']]

var_Bit_senti=pd.concat([df_h,df4],axis=1)
var_Bit_senti.to_csv("complete_var_exist_Nan.csv",index=False)
df_var_ommit_na = var_Bit_senti.dropna(how='any')
# nanがあればすべて取り除く

In [65]:
df_var_ommit_na

,time,high,low,open,volumefrom,6_hours_ago,3_hours_ago
10,2018-04-01 10:00:00,-0.003835,-0.011489,0.002870,0.267168,0.000000,0.000000
11,2018-04-01 11:00:00,-0.007720,0.000186,-0.010919,-0.159632,0.000000,0.000000
12,2018-04-01 12:00:00,0.001321,0.003349,-0.000236,-0.295588,0.000000,0.000000
13,2018-04-01 13:00:00,-0.000447,-0.001616,0.002223,0.180988,0.200000,0.125000
14,2018-04-01 14:00:00,-0.005130,-0.012716,-0.001908,0.954167,0.166667,0.100000
...,...,...,...,...,...,...,...
31435,2021-10-31 19:00:00,-0.002938,-0.001075,0.001923,-0.209830,0.333333,0.166667
31436,2021-10-31 20:00:00,-0.001648,-0.002445,-0.006050,0.114681,0.250000,0.250000
31437,2021-10-31 21:00:00,-0.002891,-0.002894,-0.002556,0.167903,0.000000,0.200000
31438,2021-10-31 22:00:00,0.001945,0.003239,0.001513,0.017864,0.000000,0.200000


In [67]:
df_var_ommit_na.to_csv("omitNan_var.csv",index=False)


In [66]:
var_Bit_senti

,time,high,low,open,volumefrom,6_hours_ago,3_hours_ago
0,2018-04-01 00:00:00,0.013297,0.004177,-0.002860,0.683997,NaN,NaN
1,2018-04-01 01:00:00,0.000096,0.001754,0.013493,-0.037853,NaN,NaN
2,2018-04-01 02:00:00,-0.004781,-0.004786,-0.004023,-0.237691,NaN,NaN
3,2018-04-01 03:00:00,-0.009421,-0.005332,-0.011961,-0.132356,NaN,NaN
4,2018-04-01 04:00:00,0.000061,-0.014133,0.003478,0.751382,NaN,NaN
...,...,...,...,...,...,...,...
31435,2021-10-31 19:00:00,-0.002938,-0.001075,0.001923,-0.209830,0.333333,0.166667
31436,2021-10-31 20:00:00,-0.001648,-0.002445,-0.006050,0.114681,0.250000,0.250000
31437,2021-10-31 21:00:00,-0.002891,-0.002894,-0.002556,0.167903,0.000000,0.200000
31438,2021-10-31 22:00:00,0.001945,0.003239,0.001513,0.017864,0.000000,0.200000


In [68]:
# ここからしたは絶対値
df5=df_abs[:-1].loc[:,['6_hours_ago','3_hours_ago']]

abs_Bit_senti=pd.concat([df_h,df5],axis=1)
abs_Bit_senti.to_csv("complete_abs_exist_Nan.csv",index=False)
df_abs_ommit_na = abs_Bit_senti.dropna(how='any')
# nanがあればすべて取り除く

In [69]:
df_abs_ommit_na

,time,high,low,open,volumefrom,6_hours_ago,3_hours_ago
10,2018-04-01 10:00:00,-0.003835,-0.011489,0.002870,0.267168,0.000000,0.000000
11,2018-04-01 11:00:00,-0.007720,0.000186,-0.010919,-0.159632,0.000000,0.000000
12,2018-04-01 12:00:00,0.001321,0.003349,-0.000236,-0.295588,0.000000,0.000000
13,2018-04-01 13:00:00,-0.000447,-0.001616,0.002223,0.180988,0.200000,0.125000
14,2018-04-01 14:00:00,-0.005130,-0.012716,-0.001908,0.954167,0.166667,0.100000
...,...,...,...,...,...,...,...
31435,2021-10-31 19:00:00,-0.002938,-0.001075,0.001923,-0.209830,0.333333,0.166667
31436,2021-10-31 20:00:00,-0.001648,-0.002445,-0.006050,0.114681,0.250000,0.250000
31437,2021-10-31 21:00:00,-0.002891,-0.002894,-0.002556,0.167903,0.000000,0.200000
31438,2021-10-31 22:00:00,0.001945,0.003239,0.001513,0.017864,0.000000,0.200000


In [70]:
abs_Bit_senti

,time,high,low,open,volumefrom,6_hours_ago,3_hours_ago
0,2018-04-01 00:00:00,0.013297,0.004177,-0.002860,0.683997,NaN,NaN
1,2018-04-01 01:00:00,0.000096,0.001754,0.013493,-0.037853,NaN,NaN
2,2018-04-01 02:00:00,-0.004781,-0.004786,-0.004023,-0.237691,NaN,NaN
3,2018-04-01 03:00:00,-0.009421,-0.005332,-0.011961,-0.132356,NaN,NaN
4,2018-04-01 04:00:00,0.000061,-0.014133,0.003478,0.751382,NaN,NaN
...,...,...,...,...,...,...,...
31435,2021-10-31 19:00:00,-0.002938,-0.001075,0.001923,-0.209830,0.333333,0.166667
31436,2021-10-31 20:00:00,-0.001648,-0.002445,-0.006050,0.114681,0.250000,0.250000
31437,2021-10-31 21:00:00,-0.002891,-0.002894,-0.002556,0.167903,0.000000,0.200000
31438,2021-10-31 22:00:00,0.001945,0.003239,0.001513,0.017864,0.000000,0.200000


In [71]:
df_abs_ommit_na.to_csv("omitNan_abs.csv",index=False)
# omit_Nanフォルダに全部ある　解析にはフォルダ以下のファイルを使う
